In [ ]:
import pandas as pd
import numpy as pd
%matplotlib inline

In [ ]:
def ratios_total_participaciones_oros(medallas):
    """
    Devuelve una serie de pandas, en orden descendente, con el
    ratio: total participaciones / total medallas de oro ganadas
    por cada país
    """
    
    ########################################
    #
    #
    #  CÓDIGO DE LA FUNCIÓN
    #
    #
    ########################################

    return (medallas.combinados.oro/medallas.combinados.participaciones).\
            map(lambda x: x if np.isfinite(x) else 0).sort_values(ascending = False)

In [10]:
def estaciones(medallas):
    """
    Devuelve una serie de pandas con la palabra 'invierno' is el país ha ganado más medallas de invierno que de
    verano o 'verano' en caso contrario
    """
    ########################################
    #
    #
    #  CÓDIGO DE LA FUNCIÓN
    #
    #
    ########################################
    
    return (medallas.invierno.total > medallas.verano.total).map({True: "invierno", False: "verano"})

In [11]:
def porcentajes(medals):
    """        
    Devuelve un dataframe con el porcentaje de medallas 
    de cada metal en lugar del número total de ellas
    
    Ejemplo:
    
        Valor original: 
        
        Bohemia 	0	1	3
        
        Esperado:   
        Bohemia     0	25	75

    """
    ########################################
    #
    #
    #  CÓDIGO DE LA FUNCIÓN
    #
    #
    ########################################
    
    for temporada in ["verano", "invierno", "combinados"]:
        for cols in medals.loc[:, temporada].columns[1:-1]:
            medals.loc[:, (temporada, cols)] = medals.loc[:, (temporada, cols)]/medals.loc[:, (temporada, "total")]*100
        
    return medals

In [ ]:
def ordenar_por_ratio(medallas):
    """
    Función que devuelve un dataframe con los países ordenados
    de forma descendente según la suma de las medallas aplicándoles
    los siguientes pesos:
        * 1 medalla de oro en verano = 5
        * 1 medalla de plata en verano = 3.5
        * 1 medalla de bronce en verano = 1.5
        * 1 medalla de oro en invierno = 4.5
        * 1 medalla de plata en invierno = 3
        * 1 medalla de bronce en invierno = 1
    """
    ########################################
    #
    #
    #  CÓDIGO DE LA FUNCIÓN
    #
    #
    ########################################
    
    # En esta función se ha tenido en cuenta el comportamiento de los diccionarios introducido en las nuevas
    # versiones de python por el cual el orden de las claves respeta el orden de introducción de los elementos
    # en el diccionario.
    # Para versiones anteriores de python que no integren este comportamiento, una posible solución sería,
    # sustituir las dos primeras líneas del código siguiente por un simple diccionario creado ad-hoc. eg:
    #    map  = {('verano', 'oro'): 5,
    #           ('verano', 'plata'): 3.5,
    #           ('verano', 'bronce'): 1.5,
    #           ('invierno', 'oro'): 4.5,
    #           ('invierno', 'plata'): 3,
    #           ('invierno', 'bronce'): 1} 
    
    sw = [t for t in medallas.columns if t[1] in ["oro", "plata", "bronce"] if t[0] not in ["combinados"]]
    dic = {key: val for (key,val) in zip(sw, [5, 3.5, 1.5, 4.5, 3, 1])}
    
    for key, val in dic.items():
        medallas.loc[:, key] = medallas.loc[:, key]*val
        
    return pd.DataFrame(medallas.loc[:, sw].sum(axis=1).sort_values(ascending=False))

In [6]:
def insertar_rankings_totales(medallas):
    """
    Returns a new dataframe with tree new colums:
    (summer, total_rank)
    (winter, total_rank)
    (combined, total_rank)
    
    Each column will have the position in the ranking of the country in total medals of the category 
    (summer, winter, combined)
    
    """
    
    ########################################
    #
    #
    #  CÓDIGO DE LA FUNCIÓN
    #
    #
    ########################################
 
    medallas = medallas[[('verano', 'total'), ('invierno', 'total'), \
                         ('combinados', 'total')]].rank(ascending=False, method='first')
    medallas.columns = [['verano', 'invierno', 'combinados' ], ['total_rank', 'total_rank', 'total_rank']]
    
    return medallas